In [1]:
# library import
import os
import time
import numpy as np
import scipy as sp
import pandas as pd
import time

from datetime import datetime
from IPython.display import clear_output

## 필요 데이터 load

In [12]:
code_df_all = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df = code_df_all
code_df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
1,HSD엔진,82740,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진",2011-01-04,12월,고영열,http://www.doosanengine.com,경상남도
2,KG케미칼,1390,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제",1989-08-25,12월,김경묵,http://www.kgchem.co.kr,울산광역시
3,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시
4,OCI,10060,기초 화학물질 제조업,"타르제품,카본블랙,무수프탈산,농약원제,석탄화학제품,정밀화학제품,플라스틱창호재 제조,판매",1985-07-09,12월,"백우석, 이우현, 김택중(3인, 각자 대표이사)",http://www.oci.co.kr,서울특별시


In [13]:
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)
code_df = code_df[['회사명','종목코드','업종']].rename(columns={'회사명':'name', '종목코드':'code', '업종':'type'})
code_df.head()

,name,code,type
0,GS글로벌,001250,상품 종합 도매업
1,HSD엔진,082740,일반 목적용 기계 제조업
2,KG케미칼,001390,기초 화학물질 제조업
3,LG이노텍,011070,전자부품 제조업
4,OCI,010060,기초 화학물질 제조업


In [22]:
aa = code_df.type
freq = np.unique(aa.values, return_counts=True)
list(map(list, zip(*freq)))

[['1차 비철금속 제조업', 20],
 ['1차 철강 제조업', 57],
 ['가구 제조업', 7],
 ['가전제품 및 정보통신장비 소매업', 1],
 ['가정용 기기 제조업', 10],
 ['가죽, 가방 및 유사제품 제조업', 5],
 ['개인 및 가정용품 임대업', 1],
 ['건물 건설업', 25],
 ['건물설비 설치 공사업', 3],
 ['건축기술, 엔지니어링 및 관련 기술 서비스업', 13],
 ['건축자재, 철물 및 난방장치 도매업', 1],
 ['경비, 경호 및 탐정업', 1],
 ['고무제품 제조업', 8],
 ['곡물가공품, 전분 및 전분제품 제조업', 8],
 ['골판지, 종이 상자 및 종이용기 제조업', 9],
 ['과실, 채소 가공 및 저장 처리업', 1],
 ['광고업', 8],
 ['교육지원 서비스업', 2],
 ['구조용 금속제품, 탱크 및 증기발생기 제조업', 20],
 ['귀금속 및 장신용품 제조업', 1],
 ['그외 기타 개인 서비스업', 1],
 ['그외 기타 운송장비 제조업', 3],
 ['그외 기타 전문, 과학 및 기술 서비스업', 8],
 ['그외 기타 제품 제조업', 7],
 ['금속 주조업', 4],
 ['금융 지원 서비스업', 70],
 ['기계장비 및 관련 물품 도매업', 23],
 ['기록매체 복제업', 2],
 ['기반조성 및 시설물 축조관련 전문공사업', 4],
 ['기초 의약물질 및 생물학적 제제 제조업', 32],
 ['기초 화학물질 제조업', 43],
 ['기타 과학기술 서비스업', 6],
 ['기타 교육기관', 1],
 ['기타 금속 가공제품 제조업', 23],
 ['기타 금융업', 87],
 ['기타 비금속 광물제품 제조업', 11],
 ['기타 비금속광물 광업', 1],
 ['기타 사업지원 서비스업', 7],
 ['기타 상품 전문 소매업', 2],
 ['기타 섬유제품 제조업', 3],
 ['기타 식품 제조업', 33],
 ['기타 운송관련 서비스업', 6],
 ['기타 전기장비 제조업', 7]

In [58]:
stock_df = pd.read_csv('일별시세.csv', encoding='euc-kr')
stock_df.code = stock_df.code.map('{:06d}'.format)

In [59]:
stock_df.head()

,name,code,date,closing_price,open_price,high_price,low_price,volume
0,GS글로벌,001250,2019-05-22,"2,630","2,650","2,660","2,595","390,300"
1,GS글로벌,001250,2019-05-21,"2,645","2,625","2,685","2,615","375,080"
2,GS글로벌,001250,2019-05-20,"2,655","2,755","2,785","2,635","447,674"
3,GS글로벌,001250,2019-05-17,"2,750","2,800","2,845","2,750","275,411"
4,GS글로벌,001250,2019-05-16,"2,785","2,845","2,855","2,770","302,705"


In [60]:
stock_df.insert(loc=2, column='type', value=pd.merge(stock_df, code_df, how='left', on=['name','code']).type, allow_duplicates=True)

In [62]:
stock_df.head()

,name,code,type,date,closing_price,open_price,high_price,low_price,volume
0,GS글로벌,001250,상품 종합 도매업,2019-05-22,"2,630","2,650","2,660","2,595","390,300"
1,GS글로벌,001250,상품 종합 도매업,2019-05-21,"2,645","2,625","2,685","2,615","375,080"
2,GS글로벌,001250,상품 종합 도매업,2019-05-20,"2,655","2,755","2,785","2,635","447,674"
3,GS글로벌,001250,상품 종합 도매업,2019-05-17,"2,750","2,800","2,845","2,750","275,411"
4,GS글로벌,001250,상품 종합 도매업,2019-05-16,"2,785","2,845","2,855","2,770","302,705"
